# **DATA & LIBRARY**

In [ ]:
import pandas as pd
import numpy as np
import random
import math
from math import sqrt
from sklearn.metrics import mean_squared_error


dataset = pd.read_csv("/content/TLKM.csv")
dataset = dataset[['date','high']]
dataset['y1'] = dataset.shift(periods=1)['high']
for i in range(10):
      dataset[f"y{i+1}"] = dataset.shift(periods=i+1)['high']
data = dataset[-200: -100]
print(data.head(100))

                    date    high      y1  ...      y8      y9     y10
215  2020-06-10T00:00:00  3190.0  3270.0  ...  3210.0  3270.0  3280.0
216  2020-06-11T00:00:00  3150.0  3190.0  ...  3150.0  3210.0  3270.0
217  2020-06-12T00:00:00  3080.0  3150.0  ...  3270.0  3150.0  3210.0
218  2020-06-15T00:00:00  3150.0  3080.0  ...  3320.0  3270.0  3150.0
219  2020-06-16T00:00:00  3220.0  3150.0  ...  3390.0  3320.0  3270.0
..                   ...     ...     ...  ...     ...     ...     ...
310  2020-10-27T00:00:00  2690.0  2680.0  ...  2850.0  2830.0  2740.0
311  2020-11-02T00:00:00  2620.0  2690.0  ...  2780.0  2850.0  2830.0
312  2020-11-03T00:00:00  2590.0  2620.0  ...  2810.0  2780.0  2850.0
313  2020-11-04T00:00:00  2640.0  2590.0  ...  2740.0  2810.0  2780.0
314  2020-11-05T00:00:00  2810.0  2640.0  ...  2710.0  2740.0  2810.0

[100 rows x 12 columns]


# **PROCEDURE & FUNGSI**

**Generate Populasi**

In [ ]:
def generate_populasi(jml_populasi, panjang) :
    populasi = []
    for i in range(jml_populasi) : 

         chromosome = [random.uniform(-100, 100) for i in range(10)]

        # Input individu ke array
         populasi.append(chromosome)

    return populasi 

**Decode Chromosome**

In [ ]:
def decode_chromosome(populasi, rmin, max):
  populasi             = np.asarray(populasi)
  pop_size, chrom_size = populasi.shape
  jumlah_x             = len(rmin)
  alele                = chrom_size // jumlah_x
  temp_populasi        = populasi.copy().reshape((pop_size, jumlah_x, alele))

  decimal              = np.array( [10**-(i+1) for i in range(alele) ])
  scale                = (max-rmin) / (9*np.sum(decimal))

  decoded_populasi     = rmin + (scale * temp_populasi.dot(decimal))

  return decoded_populasi

**Fitness**

In [ ]:
def nilai_minimum(x1,x2):
  return (math.cos(x1)*math.sin(x2)) - (x1/(x2**2+1))    

In [ ]:
def fitness(decoded_kromosom):
  fitness = []
  for data in decoded_kromosom:
    x1 = data[0]                                        
    x2 = data[1]                                        
    h = nilai_minimum(x1,x2)                            
    c = 10                                              
    f = c** -(h)                                        
    fitness.append(f)

  return np.array(fitness)

Selection

In [ ]:
def tournamentSelection(pop, k):
  best1 = 0.0                                           
  best2 = 0.0                                           
  best = []                                             
  best_index = []                                       

  for i in range(k):                                    
    index = random.randint(0, (pop.size-1))             
    if (best1 == 0.0) or pop[index] > best:            
      best1 = pop[index]                                
  best.append(best1)
  best_index.append(index)

  for j in range(k):                                    
    index = random.randint(0, (pop.size-1))             
    if (best2 == 0.0) or pop[index] > best:
      best2 = pop[index]                                
  best.append(best2)
  best_index.append(index)


  return np.array(best_index) 

**Crossover**

In [ ]:
def crossover(kromosom_parents):
  chance = random.random()                                              
  child = kromosom_parents
  if(chance <= kemungkinan_crossover):
    cross = np.random.randint(0, ukuran_kromosom)                
    child[::-1, cross:] = kromosom_parents[::, cross:]     

  return child

**Mutasi**

In [ ]:
def mutasi(kromosom):
  chance = random.random()                                                                
  mutation = np.random.randint(0,10,(kromosom.shape))                                     
  mutated_kromosom = np.where(chance >= kemungkinan_mutasi, kromosom, mutation)           

  return mutated_kromosom

In [ ]:
def elitisme(populasi, fitness):
  best_index = np.argmax(fitness)                                     
  best_kromosom = populasi[best_index]                                
  populasi_baru = np.array([best_kromosom])

  return populasi_baru

# **Main Program**

In [ ]:
ukuran_populasi = 80            
ukuran_kromosom = 8             
ukuran_turnamen = 10            
ukuran_generasi = 100           
xMin = np.array([-1, -1])       
xMax = np.array([2, 1])         
kemungkinan_crossover = 0.8     
kemungkinan_mutasi = 0.05       

populasi = generate_populasi(ukuran_populasi, ukuran_kromosom)

for currentGenerasi in range(ukuran_generasi):
  getDecode_Chrom = decode_chromosome(populasi, xMin,  xMax)
  getFitness = fitness(getDecode_Chrom)
  populasi_baru = elitisme(populasi, getFitness)
  while(len(populasi_baru) < ukuran_populasi):
    getParentsIndex = tournamentSelection(getFitness, ukuran_turnamen)
    parents = []
    parent1 = populasi[getParentsIndex[0]]
    parent2 = populasi[getParentsIndex[1]]
    parents.append(parent1)
    parents.append(parent2)
    getMutatedChild = mutasi(getCrossoveredChild)
    populasi_baru = np.vstack([populasi_baru, getMutatedChild])
  populasi = populasi_baru

decoded_populasi = decode_chromosome(populasi, xMin, xMax)
fitness = fitness(decoded_populasi)
best_index = np.argmax(fitness)

print("")
print("")
best_individu = populasi[best_index]
print("Individu fitness terbaik : ",best_individu)
best_decoded = decoded_populasi[best_index]
print("Nilai x1 dan x2                 : ",best_decoded)
print("Nilai prediksi terbaik          : ",nilai_minimum(best_decoded[0],best_decoded[1]))
print("")
print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if __name__ == '__main__':




Individu fitness terbaik :  [ 98.7692993   56.08892259  86.49826682  48.29207833 -14.12567171
  -9.18245622  90.02411385  -0.8626926  -25.38935028  69.66045447]
Nilai x1 dan x2                 :  [30.58733199 -1.04141943]
Nilai prediksi terbaik          :  -15.25671361209215


